In [1]:
import flask
from flask import Flask
import gspread
from oauth2client.service_account import ServiceAccountCredentials
import pandas as pd
import datetime
import time

In [2]:
# Connect to Google Sheets
scope = ['https://www.googleapis.com/auth/spreadsheets']

credentials = ServiceAccountCredentials.from_json_keyfile_name("grlproject-credentials.json", scope)
client = gspread.authorize(credentials)

# Mappings
Status >> Status sheet \
rules >> sheet with rules \
logs_curr >> sheet with logs of curr_year \
to_do >> todo list (without points) sheet \
recurring_tasks >> recurring task list (with points)

In [3]:
from collections import deque
class GRL:
    def __init__(self):
        # initallizing some constants and sheets access in local variables
        self.db_key = "11glFKrtVXULhoUB5h4LgQOoJlt3ckuCaI4SRX_E-eng"
        self.db_wb = client.open_by_key(self.db_key)

        curr_year = (datetime.date.today().year) % 100
        self.sheets_name = {"Status" : "Status", 
                            "logs" : "logs {}".format(curr_year), 
                            "rules" : "rules",
                            "to_do" : "to do",
                            'recurring_tasks' : "recurring"}

        self.sheets = dict([])
        for key, name in self.sheets_name.items() : 
            self.sheets[key] = self.db_wb.worksheet(name)

        # add_log is used to temporarily save logs before pushing them to sheets_db
        self.add_log = []
        
        # deque to keep track of api limits
        self.read_api_calls = deque()
        self.write_api_calls = deque()
        
        self.variables = dict({})
        
        ########### initial status pull needs to decide if same pull_status or not #################
        self.pull_status()
#         self.new_day()
        
    # keeps track of read calls of sheets api
    def read_api(self) : 
        self.read_api_calls.append(time.time())
        while len(self.read_api_calls) > 50 : 
            while (self.read_api_calls[0] + 1) < time.time() : self.read_api_calls.popleft()
            print('read_limit')
            time.sleep(2)

    # keeps track of write calls of sheets api
    def write_api(self) : 
        self.write_api_calls.append(time.time())
        while len(self.write_api_calls) > 50 : 
            while (self.write_api_calls[0] + 1) < time.time() : self.write_api_calls.popleft()
            print('write_limit')
            time.sleep(2)


    # stores info on score related updates
    def add_score(self, updates, log) :
        updates['Status'].append({'range' : 'A2', 'values' : [[int(log.split(';')[-1])]]})
        
    # stores info on dt_completed related updates
    def add_dt_completed(self, updates, log, t, v) :
        task_no = '$EFG'
        updates['Status'].append({'range' : '{}2'.format(task_no[t]), 'values' : [[v]]})
        
    # stores info on 'last time of update' in updates
    def add_time_update(self, updates) :
        updates['Status'].append({'range' : 'I2', 'values' : [[datetime.datetime.strftime(datetime.date.today(), "%Y%m%d")]]})
        
        
    # pushes all stored updates together so that api calls needed are less
    def push_all_updates(self, updates) :
        for sheet, update_list in updates.items() :
            if len(update_list) == 0 : continue
            self.sheets[sheet].batch_update(update_list)
            self.write_api()
    
    
    # handles failed attempts to write in sheets db
    def repair_sheet(self) :
        old_log_count = int(self.sheets['Status'].acell('H2').value)
        new_log_count = int(self.sheets['logs'].acell('A1').value)
        self.read_api()
        self.read_api()
        if old_log_count == new_log_count : return
        
        temp_add_log = self.sheets['logs'].get('A{}:A{}'.format(old_log_count + 2, new_log_count + 2))
        self.read_api()
        
        updates = {key : [] for key in self.sheets.keys()}
        for log in temp_add_log[0] :
            operation = log.split(';')[0]
            if operation == "UPDATE SCORE" :
                self.add_score(updates, log)
            elif operation == 'COMPLETED DT1' :
                self.add_dt_completed(updates, log, 1, 1)
            elif operation == 'COMPLETED DT2' : 
                self.add_dt_completed(updates, log, 2, 1)
            elif operation == 'COMPLETED DT3' : 
                self.add_dt_completed(updates, log, 3, 1)
            elif operation == 'UNDO DT1' :
                self.add_dt_completed(updates, log, 1, 0)
            elif operation == 'UNDO DT2' : 
                self.add_dt_completed(updates, log, 2, 0)
            elif operation == 'UNDO DT3' : 
                self.add_dt_completed(updates, log, 3, 0)
        
        ############# is it needed here????? ##########
        self.add_time_update(updates)    # updating last open date
        
        self.push_all_updates(updates)
        
        self.sheets['Status'].update('H2', new_log_count)     # updating final log count in status sheet
        self.read_api()
        
    
    # updates sheets db with current changes and logs
    def push_status(self) :
        
        if len(self.add_log) == 0 : return       # if there's no log to push, return
        
        # reading position for next empty cell in logs and pushing logs in db and updating count of logs
        cell_ptr = int(self.sheets['logs'].acell('A1').value) + 2
        self.sheets['logs'].update('A{}:A{}'.format(cell_ptr, cell_ptr + len(self.add_log)), [[log] for log in self.add_log])
        self.sheets['logs'].update('A1', cell_ptr + len(self.add_log) - 2)
        
        self.read_api()                     # tracking api calls
        self.write_api()
        self.write_api()
        
        updates = {key : [] for key in self.sheets.keys()}
        
        for log in self.add_log :
            operation = log.split(';')[0]
            if operation == "UPDATE SCORE" :
                self.add_score(updates, log)
            elif operation == 'COMPLETED DT1' :
                self.add_dt_completed(updates, log, 1, 1)
            elif operation == 'COMPLETED DT2' : 
                self.add_dt_completed(updates, log, 2, 1)
            elif operation == 'COMPLETED DT3' : 
                self.add_dt_completed(updates, log, 3, 1)
            elif operation == 'UNDO DT1' :
                self.add_dt_completed(updates, log, 1, 0)
            elif operation == 'UNDO DT2' : 
                self.add_dt_completed(updates, log, 2, 0)
            elif operation == 'UNDO DT3' : 
                self.add_dt_completed(updates, log, 3, 0)
        
        
        ############# is it needed here????? ##########
        self.add_time_update(updates)    # updating last open date
        
        self.push_all_updates(updates)
        
        self.sheets['Status'].update('H2', cell_ptr + len(self.add_log) - 2)     # updating final log count in status sheet
        self.read_api()
        
        self.add_log = []
        self.pull_status()
    
    
    # pulls status from db in as it is form in dictionary
    # necessary changes are performed in accessing variable from '.' operator  
    def pull_status(self) :
        self.repair_sheet()
        zipped_temp_sheet = zip(*self.sheets['Status'].get_values())
        self.read_api()
        
        for key, value in zipped_temp_sheet :
            self.variables[key] =  value 
        ################### fetched status till now need to see what is needed as per update functions ###############
    
    
    ############# need to complete new_day feature #####################
    # checks whether new day and performs daily tasks
    def new_day(self) :
        _today = datetime.datetime.strftime(datetime.date.today(), "%Y%m%d")
        if self.variables['Last open'] == _today : return
        
        daily_charge = {'S' : 500, 'A' : 700, 'B' : 800, 'C' : 900, 'D' : 1000}
        
        self.add_log = ['NEW DAY;' + _today + ';DAILY CHARGE;']
        old_score = self.score
        new_score = self.score
        new_score -= daily_charge[self.membership]
        self.add_log.append('RESET DT;')
        self.add_log.append('UPDATE SCORE;' + str(old_score) + ';' + str(new_score))
        self.push_status()
    
    @property
    def score(self) :
        return int(self.variables['Score'])
    
    @property
    def membership(self) :
        if self.score < 1e6 : return 'D'
        elif self.score < 1e9 : return 'C'
        elif self.score < 1e12 : return 'B'
        elif self.score < 1e15 : return 'A'
        else : return 'S'
    
    
    
    @property
    def dt1(self) :    return self.variables['DT1']
    
    @property
    def dt2(self) :    return self.variables['DT2']
    
    @property
    def dt3(self) :    return self.variables['DT3']
    
    
    
    @property
    def dt1_done(self) :    return int(self.variables['DT1_completed'])
    
    @dt1_done.setter
    def dt1_done(self, value) :
        if self.dt1_done == value : return                     # really needed ?? or needs to be modified ??
        t_done = self.dt1_done + self.dt2_done + self.dt3_done
        old_score = self.score
        new_score = self.score
        
        if value == 1 : 
            self.add_log = ["COMPLETED DT1;" + self.dt1 + ";"]
            if t_done == 0 : new_score += 200
            elif t_done == 1 : new_score += 300
            else : new_score += 500
        else :
            self.add_log = ["UNDO DT1;" + self.dt1 + ";"]
            if t_done == 1 : new_score -= 200
            elif t_done == 2 : new_score -= 300
            else : new_score -= 500
        
        self.add_log.append('UPDATE SCORE;' + str(old_score) + ';' + str(new_score))
        self.push_status()
         
    @property
    def dt2_done(self) :    return int(self.variables['DT2_completed'])
    
    @dt2_done.setter
    def dt2_done(self, value) :
        if self.dt2_done == value : return                     # really needed ?? or needs to be modified ??
        t_done = self.dt1_done + self.dt2_done + self.dt3_done
        old_score = self.score
        new_score = self.score
        
        if value == 1 : 
            self.add_log = ["COMPLETED DT2;" + self.dt2 + ";"]
            if t_done == 0 : new_score += 200
            elif t_done == 1 : new_score += 300
            else : new_score += 500
        else :
            self.add_log = ["UNDO DT2;" + self.dt2 + ";"]
            if t_done == 1 : new_score -= 200
            elif t_done == 2 : new_score -= 300
            else : new_score -= 500
        
        self.add_log.append('UPDATE SCORE;' + str(old_score) + ';' + str(new_score))
        self.push_status()
    
    @property
    def dt3_done(self) :    return int(self.variables['DT3_completed'])
    
    @dt3_done.setter
    def dt3_done(self, value) :
        if self.dt3_done == value : return                     # really needed ?? or needs to be modified ??
        t_done = self.dt1_done + self.dt2_done + self.dt3_done
        old_score = self.score
        new_score = self.score
        
        if value == 1 : 
            self.add_log = ["COMPLETED DT3;" + self.dt3 + ";"]
            if t_done == 0 : new_score += 200
            elif t_done == 1 : new_score += 300
            else : new_score += 500
        else :
            self.add_log = ["UNDO DT3;" + self.dt3 + ";"]
            if t_done == 1 : new_score -= 200
            elif t_done == 2 : new_score -= 300
            else : new_score -= 500
        
        self.add_log.append('UPDATE SCORE;' + str(old_score) + ';' + str(new_score))
        self.push_status()
    
#     # not a property because in future we may have it like top 10 or 5 only, also updates will be complicated
#     def get_to_do_list(self) :
#         return self.sheets['to_do'].col_values(1)
    
#     # not a property because in future we may have it like top 10 or 5 only, also updates will be complicated
#     def get_recurring_tasks(self) :
#         tasks = self.sheets['recurring_tasks'].col_values(1)
#         points = self.sheets['recurring_tasks'].col_values(2)
#         return dict(zip(tasks, points))

In [4]:
player = GRL() 

In [ ]:
app = Flask(__name__)

@app.route('/', methods=['POST', 'GET'])
def home():
    print(flask.request.form)
    if flask.request.method == 'POST' :
        if 'DT_B1' in flask.request.form : player.dt1_done = 1 - player.dt1_done
        if 'DT_B2' in flask.request.form : player.dt2_done = 1 - player.dt2_done
        if 'DT_B3' in flask.request.form : player.dt3_done = 1 - player.dt3_done
        return flask.redirect(flask.url_for('home'))

    variables = {'score' : player.score,
                 'membership' : player.membership,
                 'DT1' : player.dt1,
                 'DT2' : player.dt2,
                 'DT3' : player.dt3,
                 'DT1_done' : player.dt1_done,
                 'DT2_done' : player.dt2_done,
                 'DT3_done' : player.dt3_done,
#                  'to_do' : player.get_to_do_list(),
#                  'recurring_tasks' : player.get_recurring_tasks()
                }
    
    return flask.render_template("index.html", **variables)

app.run()

 * Serving Flask app '__main__' (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://127.0.0.1:5000/ (Press CTRL+C to quit)
127.0.0.1 - - [23/Dec/2022 15:01:39] "GET / HTTP/1.1" 200 -


ImmutableMultiDict([])


In [200]:
class GRL:
    def __init__(self):
        self.db_key = "11glFKrtVXULhoUB5h4LgQOoJlt3ckuCaI4SRX_E-eng"
        self.db_wb = client.open_by_key(self.db_key)

        curr_year = (datetime.date.today().year) % 100
        self.sheets_name = {"Status" : "Status", 
                            "logs" : "logs {}".format(curr_year), 
                            "rules" : "rules",
                            "to_do" : "to do",
                            'recurring_tasks' : "recurring"}

        self.sheets = dict([])
        for key, name in self.sheets_name.items() : 
            self.sheets[key] = self.db_wb.worksheet(name)

        self.add_log = []
        
    # pulls status from db
    def pull_status(self) :
        
    
    # handles failed attempts to write in sheets database
    def repair(self) :
        old_log_count = int(self.sheets['Status'].acell('H2').value)
        new_log_count = int(self.sheets['logs'].acell('A1').value)
        if old_log_count == new_log_count : return
        
        add_log = self.sheets['logs'].get('A{}:A{}'.format(old_log_count + 2, new_log_count + 2))
        for log in add_log :
            operation = log.split(';')[0]
            if operation == "UPDATE SCORE" :
                self.sheets['Status'].update('A2', int(log.split(';')[-1]))
            elif self.dt1_done == 0 and operation == 'COMPLETED DT1' : 
                self.sheets['Status'].update('E2', int(self.sheets['Status'].acell('E2').value) + 1)
            elif self.dt2_done == 0 and operation == 'COMPLETED DT2' : 
                self.sheets['Status'].update('E2', int(self.sheets['Status'].acell('E2').value) + 2)
            elif self.dt3_done == 0 and operation == 'COMPLETED DT3' : 
                self.sheets['Status'].update('E2', int(self.sheets['Status'].acell('E2').value) + 4)
            elif self.dt1_done == 1 and operation == 'UNDO DT1' : 
                self.sheets['Status'].update('E2', int(self.sheets['Status'].acell('E2').value) - 1)
            elif self.dt2_done == 1 and operation == 'UNDO DT2' : 
                self.sheets['Status'].update('E2', int(self.sheets['Status'].acell('E2').value) - 2)
            elif self.dt3_done == 1 and operation == 'UNDO DT3' : 
                self.sheets['Status'].update('E2', int(self.sheets['Status'].acell('E2').value) - 4)
        
        self.sheets['Status'].update('G2', datetime.datetime.strftime(datetime.date.today(), "%Y%m%d"))
        
        self.sheets['Status'].update('H2', new_log_count)
    
    
    # updates sheets database with current changes and logs
    def push_status(self) :
        self.repair()
        
        if len(self.add_log) == 0 : return
        
        cell_ptr = int(self.sheets['logs'].acell('A1').value) + 2
        self.sheets['logs'].update('A{}:A{}'.format(cell_ptr, cell_ptr + len(self.add_log)), [[log] for log in self.add_log])
        self.sheets['logs'].update('A1', cell_ptr + len(self.add_log) - 2)
        
        for log in self.add_log :
            operation = log.split(';')[0]
            if operation == "UPDATE SCORE" :
                self.sheets['Status'].update('A2', int(log.split(';')[-1]))
            elif self.dt1_done == 0 and operation == 'COMPLETED DT1' : 
                self.sheets['Status'].update('E2', int(self.sheets['Status'].acell('E2').value) + 1)
            elif self.dt2_done == 0 and operation == 'COMPLETED DT2' : 
                self.sheets['Status'].update('E2', int(self.sheets['Status'].acell('E2').value) + 2)
            elif self.dt3_done == 0 and operation == 'COMPLETED DT3' : 
                self.sheets['Status'].update('E2', int(self.sheets['Status'].acell('E2').value) + 4)
            elif self.dt1_done == 1 and operation == 'UNDO DT1' : 
                self.sheets['Status'].update('E2', int(self.sheets['Status'].acell('E2').value) - 1)
            elif self.dt2_done == 1 and operation == 'UNDO DT2' : 
                self.sheets['Status'].update('E2', int(self.sheets['Status'].acell('E2').value) - 2)
            elif self.dt3_done == 1 and operation == 'UNDO DT3' : 
                self.sheets['Status'].update('E2', int(self.sheets['Status'].acell('E2').value) - 4)
        
        self.sheets['Status'].update('G2', datetime.datetime.strftime(datetime.date.today(), "%Y%m%d"))
            
        self.sheets['Status'].update('H2', cell_ptr + len(self.add_log) - 2)
        
        self.add_log = []
    
    @property
    def score(self) :
        return int(self.sheets['Status'].acell('A2').value)
    
    
    @property
    def membership(self) :
        if self.score < 1e6 : return 'D'
        elif self.score < 1e9 : return 'C'
        elif self.score < 1e12 : return 'B'
        elif self.score < 1e15 : return 'A'
        else : return 'S'
    
    
    @property
    def dt1(self) :
        return self.sheets['Status'].acell('B2').value
    
    @property
    def dt2(self) :
        return self.sheets['Status'].acell('C2').value
    
    @property
    def dt3(self) :
        return self.sheets['Status'].acell('D2').value
    
    
    @property
    def dt1_done(self) :
        return (int(self.sheets['Status'].acell('E2').value) % 2)
    
    @dt1_done.setter
    def dt1_done(self, value) :
        if player.dt1_done == value : return
        t_done = self.dt1_done + self.dt2_done + self.dt3_done
        old_score = self.score
        new_score = self.score
        
        if value == 1 : 
            self.add_log = ["COMPLETED DT1;" + self.dt1 + ";"]
            if t_done == 0 : new_score += 200
            elif t_done == 1 : new_score += 300
            else : new_score += 500
        else :
            self.add_log = ["UNDO DT1;" + self.dt1 + ";"]
            if t_done == 1 : new_score -= 200
            elif t_done == 2 : new_score -= 300
            else : new_score -= 500
        
        self.add_log.append('UPDATE SCORE;' + str(old_score) + ';' + str(new_score))
        self.push_status()
    
    @property
    def dt2_done(self) :
        return (int(self.sheets['Status'].acell('E2').value) % 4) // 2
    
    @dt2_done.setter
    def dt2_done(self, value) :
        if player.dt2_done == value : return
        t_done = self.dt1_done + self.dt2_done + self.dt3_done

        old_score = self.score
        new_score = self.score
        
        if value == 1 : 
            self.add_log = ["COMPLETED DT2;" + self.dt2 + ";"]
            if t_done == 0 : new_score += 200
            elif t_done == 1 : new_score += 300
            else : new_score += 500
        else :
            self.add_log = ["UNDO DT2;" + self.dt2 + ";"]
            if t_done == 1 : new_score -= 200
            elif t_done == 2 : new_score -= 300
            else : new_score -= 500
        
        self.add_log.append('UPDATE SCORE;' + str(old_score) + ';' + str(new_score))
        self.push_status()
    
    @property
    def dt3_done(self) :
        return (int(self.sheets['Status'].acell('E2').value) // 4)
    
    @dt3_done.setter
    def dt3_done(self, value) :
        if player.dt3_done == value : return
        t_done = self.dt1_done + self.dt2_done + self.dt3_done

        old_score = self.score
        new_score = self.score
        
        if value == 1 : 
            self.add_log = ["COMPLETED DT3;" + self.dt3 + ";"]
            if t_done == 0 : new_score += 200
            elif t_done == 1 : new_score += 300
            else : new_score += 500
        else :
            self.add_log = ["UNDO DT3;" + self.dt3 + ";"]
            if t_done == 1 : new_score -= 200
            elif t_done == 2 : new_score -= 300
            else : new_score -= 500
        
        self.add_log.append('UPDATE SCORE;' + str(old_score) + ';' + str(new_score))
        self.push_status()
    
    # not a property because in future we may have it like top 10 or 5 only, also updates will be complicated
    def get_to_do_list(self) :
        return self.sheets['to_do'].col_values(1)
    
    # not a property because in future we may have it like top 10 or 5 only, also updates will be complicated
    def get_recurring_tasks(self) :
        tasks = self.sheets['recurring_tasks'].col_values(1)
        points = self.sheets['recurring_tasks'].col_values(2)
        return dict(zip(tasks, points))

IndentationError: expected an indented block after function definition on line 20 (1644002198.py, line 24)